In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

In [2]:
# df = pd.read_csv('./datasets/modis_2014_United_States.csv')
# df

In [3]:
# for year in range(2015, 2025):
#     append_df = pd.read_csv(f"./datasets/modis_{year}_United_States.csv")
#     df = pd.concat([df, append_df], ignore_index=True)

In [4]:
# df.to_csv("wildfires.csv", index=False)

In [5]:
# wildfires = pd.read_csv('./datasets/wildfires.csv')
# wildfires

In [6]:
# california = gpd.read_file(
#     "https://raw.githubusercontent.com/glynnbird/usstatesgeojson/master/california.geojson"
# )

# gdf = gpd.GeoDataFrame(
#     wildfires,
#     geometry=gpd.points_from_xy(wildfires.longitude, wildfires.latitude),
#     crs="EPSG:4326"
# )

# wildfires_ca = gdf[gdf.within(california.geometry.iloc[0])]

In [8]:
# wildfires_ca.to_csv("wildfires_ca.csv", index=False)

In [11]:
wildfires_ca = pd.read_csv('./datasets/wildfires_ca.csv')
wildfires_ca

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type,geometry
0,39.3657,-121.5311,301.0,1.6,1.3,2014-01-01,1927,Terra,MODIS,22,6.20,287.7,8.5,D,0,POINT (-121.5311 39.3657)
1,39.3547,-121.5348,305.7,1.6,1.3,2014-01-01,1927,Terra,MODIS,62,6.20,287.8,14.5,D,0,POINT (-121.5348 39.3547)
2,39.3584,-121.5533,302.9,1.6,1.3,2014-01-01,1927,Terra,MODIS,54,6.20,286.5,10.9,D,0,POINT (-121.5533 39.3584)
3,40.0706,-121.7488,302.7,2.1,1.4,2014-01-03,537,Terra,MODIS,50,6.20,279.3,25.1,N,0,POINT (-121.7488 40.0706)
4,40.0590,-121.7416,310.7,2.1,1.4,2014-01-03,537,Terra,MODIS,81,6.20,279.4,39.0,N,0,POINT (-121.7416 40.059)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196491,33.5558,-116.6458,309.2,1.2,1.1,2024-12-29,2139,Aqua,MODIS,60,61.03,298.1,7.1,D,0,POINT (-116.6458 33.5558)
196492,32.6190,-116.1323,312.7,1.1,1.1,2024-12-29,2139,Aqua,MODIS,63,61.03,299.4,7.6,D,0,POINT (-116.1323 32.619)
196493,35.5633,-119.1824,305.6,1.5,1.2,2024-12-30,2219,Aqua,MODIS,54,61.03,286.0,13.6,D,2,POINT (-119.1824 35.5633)
196494,35.5690,-119.1734,302.3,1.5,1.2,2024-12-30,2219,Aqua,MODIS,43,61.03,285.4,9.5,D,0,POINT (-119.1734 35.569)
